In [163]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow_io as tfio
from pathlib import Path

from sklearn.preprocessing import MultiLabelBinarizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten

**Build Data Loading Function**

In [164]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

**Create Tensorflow Dataset**

In [165]:
train_dir = Path('train_data/train/')
test_dir = Path('train_data/test/')

In [166]:
def transform(value):
    if value == "Parsed_Capuchinbird_Clips":
        return tf.one_hot(0,2)
    if value == "Parsed_Not_Capuchinbird_Clips":
        return tf.one_hot(1,2)

In [167]:
train_filepaths = pd.Series(list(train_dir.glob(r'**/*.wav'))).astype(str)
train_animals = pd.Series(train_filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1])).astype(str)

train_lables = train_animals.map(transform).to_list()

train = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(train_filepaths), tf.data.Dataset.from_tensor_slices(train_lables)))

In [168]:
test_filepaths = pd.Series(list(train_dir.glob(r'**/*.wav'))).astype(str)
test_animals = pd.Series(test_filepaths.apply(lambda x: os.path.split(os.path.split(x)[0])[1])).astype(str)

test_lables = test_animals.map(transform).to_list()

test = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(test_filepaths), tf.data.Dataset.from_tensor_slices(test_lables)))

**Build Preprocessing Function to Convert to Spectrogram**

In [169]:
def preprocess(file_path, label): 
    wav = load_wav_16k_mono(file_path)
    wav = wav[:48000]
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    
    return spectrogram, label

**Create Training and Testing Partitions**

In [170]:
train = train.map(preprocess)
train = train.cache()
train = train.shuffle(buffer_size=1000)
train = train.batch(16)
train = train.prefetch(8)

test = test.map(preprocess)
test = test.cache()
test = test.shuffle(buffer_size=1000)
test = test.batch(16)
test = test.prefetch(8)

**Build Model**

In [171]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(1491, 257,1)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [172]:
model.compile('Adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [173]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 1489, 255, 16)     160       
                                                                 
 conv2d_3 (Conv2D)           (None, 1487, 253, 16)     2320      
                                                                 
 flatten_1 (Flatten)         (None, 6019376)           0         
                                                                 
 dense_2 (Dense)             (None, 128)               770480256 
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                                 
Total params: 770,482,994
Trainable params: 770,482,994
Non-trainable params: 0
_________________________________________________________________


**Train**

In [174]:
hist = model.fit(train, epochs=4, validation_data=test)

Epoch 1/4
36/36 [==============================] - 190s 5s/step - loss: 6.1852 - recall_1: 0.8728 - precision_1: 0.8728 - val_loss: 0.1445 - val_recall_1: 0.9276 - val_precision_1: 0.9276
Epoch 2/4
36/36 [==============================] - 184s 5s/step - loss: 0.0713 - recall_1: 0.9788 - precision_1: 0.9788 - val_loss: 0.0344 - val_recall_1: 0.9929 - val_precision_1: 0.9929
Epoch 3/4
36/36 [==============================] - 185s 5s/step - loss: 0.0162 - recall_1: 0.9947 - precision_1: 0.9947 - val_loss: 0.0114 - val_recall_1: 0.9965 - val_precision_1: 0.9965
Epoch 4/4
36/36 [==============================] - 192s 5s/step - loss: 0.0173 - recall_1: 0.9965 - precision_1: 0.9965 - val_loss: 8.1835e-04 - val_recall_1: 1.0000 - val_precision_1: 1.0000


In [175]:
plt.title('Loss')
plt.plot(hist.history['loss'], 'r')
plt.plot(hist.history['val_loss'], 'b')
plt.show()

**Test Model with a Single Clip**

In [178]:
X_test, y_test = test.as_numpy_iterator().next()
y_test

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [184]:
yhat = model.predict(X_test)
output = list()
for y in yhat:
    if y[0] > 0.5:
        y[0] = 1
    else:
        y[0] = 0
    if y[1] > 0.5:
        y[1] = 1
    else:
        y[1] = 0
    output.append(list(y))
output

[[0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0]]